# Common

In [1]:
import pandas as pd
import math
!pip install anytree
from anytree import Node, RenderTree

     |████████████████████████████████| 51kB 3.6MB/s 


# Load Data

In [107]:
def get_train_test_data(debug_N = None, get_all=False):
  data = pd.read_excel('/content/drive/MyDrive/ML_Hw1/data.xls')
  # data.head()

  from sklearn.model_selection import train_test_split
  data['Fedu'] = data['Fedu'].astype(str)
  data['Medu'] = data['Medu'].astype(str)
  data = data.drop_duplicates(subset=data.drop(['Label'], axis=1).columns, keep='first')
  print(f'Data_size = {data.shape[0]}')
  X = data.drop(['Label'], axis=1)
  y = data['Label']
  if get_all:
    X.reset_index(drop=True, inplace=True)
    y.reset_index(drop=True, inplace=True)
    return X, y
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)
  X_train.reset_index(drop=True, inplace=True)
  X_test.reset_index(drop=True, inplace=True)
  y_train.reset_index(drop=True, inplace=True)
  y_test.reset_index(drop=True, inplace=True)
  print(f'Train size = {X_train.shape[0]}')

  if debug_N != None:
    return X_train.iloc[:debug_N, :], X_test, y_train.iloc[:debug_N], y_test
  else:
    return X_train, X_test, y_train, y_test


# ID5

In [140]:
class Id5_Node:

  def __init__(self):
    self.has_init = False
    self.remain_attributes = []
    self.attribute = None
    self.pred = None
    self.data_x = None
    self.data_y = None
    self.depth = 0
    self.childs = {}
    self.parent = None

  def train(self, X_train, y_train):
    if self.has_init == False:
      self.remain_attributes = X_train.keys().tolist()
      self.data_x = X_train
      self.data_y = pd.DataFrame({'Label': [y_train]}, index=X_train.index)
      self.has_init = True
    else:
      self.add_sample(X_train, y_train)


    self.update_attribute(X_train, y_train)
    return

  def add_sample(self, X, y):

    self.data_x = pd.concat([self.data_x, X], axis=0)
    self.data_y.loc[X.index[0]] = y

  def update_attribute(self, X_train=None, y_train=None):
    attr_new = self.find_best_attribute()
    if(attr_new == None):
      return
    if self.attribute == None:
      self.attribute = attr_new
      self.expand()
    elif self.attribute != attr_new:
      self.pull_up(attr_new)
      self.propagate(X_train, y_train)
    else:
      self.propagate(X_train, y_train)

    return

  def propagate(self, X_train, y_train):
    sample_attr_value = X_train[self.attribute].iloc[0]

    if (sample_attr_value in self.childs.keys()):
      child = self.childs[sample_attr_value]
      child.add_sample(X_train, y_train) # X_train and y_train have one row.
      child.update_attribute(X_train, y_train)
    else:
      self.add_child(X_train,
                     pd.DataFrame({'Label': [y_train]}, index=X_train.index),
                                  sample_attr_value
                     )

    return

  def after_split_entropy(self, attr):
    data = pd.concat([self.data_x, self.data_y], axis=1)
    grouped_data = data.groupby([attr]) # attr is a string
    grouped_data_count = grouped_data.count()
    group_names = grouped_data.apply(lambda x: x)[attr].unique()
    result = 0
    for group_name in group_names:
      new_data = data[data[attr] == group_name]
      pos = new_data.loc[data['Label'] == 'yes', :].shape[0]
      neg = new_data.loc[data['Label'] == 'no', :].shape[0]
      branch_count = grouped_data_count.loc[group_name, :][0]
      result += (branch_count / data.shape[0]) * self.entropy(pos, neg)
    return result

  def find_best_attribute(self):
    if (self.data_y.shape[0] == self.data_y[self.data_y['Label'] == 'yes'].shape[0]):
      self.pred = 1
      return None
    if (self.data_y.shape[0] == self.data_y[self.data_y['Label'] == 'no'].shape[0]):
      self.pred = 0
      return None

    best_attr = self.attribute
    min_entropy = self.after_split_entropy(self.attribute) if self.attribute != None else 2
    for attribute in self.remain_attributes:
      ent = self.after_split_entropy(attribute)
      # print(f'{attribute} -> {ent}')
      if ent < min_entropy:
        min_entropy = ent
        best_attr = attribute
    return best_attr

  def entropy(self, pos, neg):
    if (pos == 0) or (neg == 0):
      return 0

    total = pos + neg
    pos_res = (pos / total) * math.log2(pos / total)
    neg_res = (neg / total) * math.log2(neg / total)

    return -(pos_res + neg_res)
      
  def expand(self, attribute=None):
    df = pd.concat([self.data_x, self.data_y],  axis=1)
    if attribute != None:
      self.attribute = attribute
    
    attr = self.attribute
    self.pred = None
    for name, group in df.groupby([attr]):
      self.add_child(child_data_x=group.drop(['Label'], axis=1),
                     child_data_y=group['Label'].to_frame(), branch_name=name
                     )
    return

  def add_child(self, child_data_x, child_data_y, branch_name):
    node = Id5_Node()
    node.data_x = child_data_x
    node.data_y = child_data_y
    node.depth = self.depth + 1
    node.remain_attributes = self.remain_attributes.copy()
    node.remain_attributes.remove(self.attribute)
    node.has_init = True
    node.parent = self
    node.update_attribute()
    self.childs[branch_name] = node
    return

  def pull_up(self, attribute):
    if self.attribute == attribute:
      return

    if self.attribute == None:
      self.expand(attribute)
      return

    for value, child in self.childs.items():
      child.pull_up(attribute)

    trees_to_merge = []
    for branch_name, child in self.childs.items():
      for child_branch_name, child_child in child.childs.items():
        self.make_temp_root(child, child_child, branch_name, child_branch_name)
      trees_to_merge.append(child)
    
    new_root = Id5_Node.merge(trees_to_merge)
    self.has_init = True
    self.remain_attributes = new_root.remain_attributes
    self.attribute = new_root.attribute
    self.pred = None
    self.depth = new_root.depth
    self.childs = new_root.childs
    self.parent = new_root.parent

    return

  def make_temp_root(self, child, child_child, branch_name, child_branch_name):
    node = Id5_Node()
    node.attribute = self.attribute
    node.depth = self.depth + 1
    node.remain_attributes = self.remain_attributes.copy()
    node.remain_attributes.remove(child.attribute)
    node.childs[branch_name] = child_child
    node.parent = child
    node.data_x = child_child.data_x
    node.data_y = child_child.data_y

    child.parent = self.parent
    child.childs[child_branch_name] = node
    child.depth = self.depth
    child.remain_attributes = self.remain_attributes.copy()

    child_child.parent = node

    # return child

  def merge(trees):
    result = trees[0]

    # get all branches
    all_branches = []
    for i in range(len(trees)):
      all_branches += list(trees[i].childs.keys())
    all_branches = list(set(all_branches))

    # merge
    for tree in trees[1:]:
      result.data_x = pd.concat([result.data_x, tree.data_x], axis=0)
      result.data_y = pd.concat([result.data_y, tree.data_y], axis=0)

      for branch_name in all_branches:
        if branch_name in tree.childs.keys():
          if branch_name in result.childs.keys():


            result.childs[branch_name].data_x = pd.concat(
                [result.childs[branch_name].data_x,
                 tree.childs[branch_name].data_x], axis=0
                 )
            
            
            result.childs[branch_name].data_y = pd.concat(
                [result.childs[branch_name].data_y,
                 tree.childs[branch_name].data_y], axis=0
                 )
            

          else:
            result.childs[branch_name] = tree.childs[branch_name]
            result.childs[branch_name].parent = result

          child_branch_name = list(tree.childs[branch_name].childs.keys())[0]

          result.childs[branch_name].childs[child_branch_name] = tree.childs[branch_name].childs[child_branch_name]
          result.childs[branch_name].childs[child_branch_name].parent = result.childs[branch_name]


    # Contraction
    for child in result.childs.values():
      if (child.data_y.shape[0] == child.data_y[child.data_y['Label'] == 'yes'].shape[0]):
        child.pred = 1
        child.attribute = None
        child.childs = {}
        continue

      if (child.data_y.shape[0] == child.data_y[child.data_y['Label'] == 'no'].shape[0]):
        child.pred = 0
        child.attribute = None
        child.childs = {}
        continue

    return result

  def make_tree_for_print(root, parent_node=None):
    if root.attribute == None:
      if(root.pred == 0):
        label = '-'
      else:
        label = '+'

      return Node(label, parent_node)

    root_node = Node(root.attribute, parent=parent_node)
    for branch_name in root.childs.keys():
      branch_node = Node(branch_name, parent=root_node)
      Id5_Node.make_tree_for_print(root.childs[branch_name], branch_node)
    
    return root_node

  def print_tree(self):
    root_node = Id5_Node.make_tree_for_print(self)
    for pre, fill, node in RenderTree(root_node):
      print("%s%s" % (pre, node.name))


  def predict_all(self, X_test, y_test):
    prediction = []
    for_index = 0
    for idx, row in X_test.iterrows():
      prediction.append(self.predict(row.to_frame().T, y_test.iloc[for_index]))
      for_index += 1
    return prediction

  def predict(self, X, y):
    if (self.attribute is None):
      return 'yes' if self.pred == 1 else 'no'

    if (X[self.attribute].iloc[0] not in self.childs.keys()):
      return 'yes' if self.data_y[self.data_y['Label'] == 'yes'].shape[0] > self.data_y[self.data_y['Label'] == 'no'].shape[0] else 'no'

    return self.childs[X[self.attribute].iloc[0]].predict(X, y)

  def prune(self, depth=5):
    if ((self.attribute is None) or (self.depth==depth-1)):
      self.pred = 1 if self.data_y[self.data_y['Label'] == 'yes'].shape[0] > self.data_y[self.data_y['Label'] == 'no'].shape[0] else 0
      self.attribute = None
      self.childs = {}
      return

    for child in self.childs.values():
      child.prune(depth)



In [138]:
class Id5_Tree:

  def __init__(self):
    self.root = Id5_Node()

  def train(self, X_train, y_train):
    for_index = 0
    for index, row in X_train.iterrows():
      print(f'Sample {index + 1}')
      try:
        self.root.train(row.to_frame().T, y_train.iloc[for_index])
      except:
        pass
      for_index += 1

  def predict_all(self, X_test, y_test):
    return self.root.predict_all(X_test, y_test)

  def print_tree(self):
    self.root.print_tree()

  def prune_tree(self, depth=5):
    self.root.prune(depth)


# Train And Print

In [8]:
pd.concat([X_train, y_train], axis=1)

,school,sex,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,Label
0,GP,F,U,GT3,T,4,2,services,other,course,mother,yes
1,GP,M,U,LE3,T,4,4,health,services,course,father,yes
2,GP,M,U,GT3,A,3,2,services,other,course,other,yes
3,GP,F,U,GT3,T,4,3,teacher,other,other,mother,yes
4,GP,F,U,GT3,T,4,3,services,other,reputation,mother,yes
...,...,...,...,...,...,...,...,...,...,...,...,...
275,GP,F,R,GT3,T,4,3,teacher,other,reputation,mother,no
276,GP,F,R,GT3,T,2,1,other,other,reputation,mother,no
277,GP,F,R,GT3,T,1,1,other,other,reputation,mother,no
278,GP,F,U,GT3,T,2,1,other,other,course,other,no


In [132]:
# Main
tree = Id5_Tree()
X_train, X_test, y_train, y_test = get_train_test_data(debug_N=None)
tree.train(X_train, y_train)
tree.print_tree()

Data_size = 374
Train size = 280
Sample 1
Sample 2
Sample 3
Sample 4
Sample 5
Sample 6
Sample 7
Sample 8
Sample 9
Sample 10
Sample 11
Sample 12
Sample 13
Sample 14
Sample 15
Sample 16
Sample 17
Sample 18
Sample 19
Sample 20
Sample 21
Sample 22
Sample 23
Sample 24
Sample 25
Sample 26
Sample 27
Sample 28
Sample 29
Sample 30
Sample 31
Sample 32
Sample 33
Sample 34
Sample 35
Sample 36
Sample 37
Sample 38
Sample 39
Sample 40
Sample 41
Sample 42
Sample 43
Sample 44
Sample 45
Sample 46
Sample 47
Sample 48
Sample 49
Sample 50
Sample 51
Sample 52
Sample 53
Sample 54
Sample 55
Sample 56
Sample 57
Sample 58
Sample 59
Sample 60
Sample 61
Sample 62
Sample 63
Sample 64
Sample 65
Sample 66
Sample 67
Sample 68
Sample 69
Sample 70
Sample 71
Sample 72
Sample 73
Sample 74
Sample 75
Sample 76
Sample 77
Sample 78
Sample 79
Sample 80
Sample 81
Sample 82
Sample 83
Sample 84
Sample 85
Sample 86
Sample 87
Sample 88
Sample 89
Sample 90
Sample 91
Sample 92
Sample 93
Sample 94
Sample 95
Sample 96
Sample 97
Sample

In [ ]:
# Test
x={'height':['short','tall','tall','tall','short','tall','tall','short'],
 'hair':['blond','dark','blond','dark','dark','red','blond','blond'],
 'eyes':['brown','brown','blue','blue','blue','blue','brown','blue']}
y={'Label':['no','no','yes','no','no','yes','no','yes']}

N = 8

test_tree = Id5_Tree()
x=pd.DataFrame(x)
y=pd.Series(['no','no','yes','no','no','yes','no','yes'])
test_tree.train(x.iloc[:N], y)
test_tree.print_tree()

# Test

In [133]:
import numpy as np

def accuracy(y_test, y_pred):
  return np.sum(y_test==y_pred) / y_test.shape[0]

def test_eval_tree(tree, X_test, y_test):
  y_pred = tree.predict_all(X_test, y_test)
  acc = accuracy(np.array(y_test.to_list()), np.array(y_pred))
  return y_pred, acc

y_pred, acc = test_eval_tree(tree, X_test, y_test)
print(f'acc= {acc*100}%')

acc= 57.446808510638306%


In [134]:
tree.prune_tree(5)
tree.print_tree()

Fedu
├── 1
│   └── Fjob
│       ├── other
│       │   └── sex
│       │       ├── F
│       │       │   └── reason
│       │       │       ├── course
│       │       │       │   └── -
│       │       │       ├── home
│       │       │       │   └── -
│       │       │       ├── other
│       │       │       │   └── -
│       │       │       └── reputation
│       │       │           └── -
│       │       └── M
│       │           └── Medu
│       │               ├── 1
│       │               │   └── -
│       │               ├── 4
│       │               │   └── -
│       │               ├── 2
│       │               │   └── -
│       │               └── 3
│       │                   └── -
│       ├── services
│       │   └── Medu
│       │       ├── 1
│       │       │   └── -
│       │       ├── 2
│       │       │   └── Mjob
│       │       │       ├── services
│       │       │       │   └── -
│       │       │       ├── at_home
│       │       │       │   └── +
│       │       │  

In [135]:
y_pred_prune, acc_prune = test_eval_tree(tree, X_test, y_test)
print(f'acc_prune= {acc_prune*100}%')

acc_prune= 61.702127659574465%


In [ ]:
stack = np.hstack((np.array(y_test.to_list()).reshape(-1, 1), np.array(y_pred).reshape(-1, 1)))
print(stack)

In [136]:
from sklearn.model_selection import KFold

def kFold_cross_validation(X, Y):

  # Cross-Validate
  kf = KFold(5, shuffle=True, random_state=1) # Use for KFold
      
  fold_accs = []

  fold = 0
  for train, test in kf.split(X):
    fold += 1
    print(f"Fold #{fold}")
    print(f'Fold Size={train.shape[0]}')
    # print(train)

    x_train = X.loc[train]
    # x_train.reset_index(drop=True, inplace=True)
    y_train = Y.loc[train]
    # y_train.reset_index(drop=True, inplace=True)
    x_test = X.loc[test]
    y_test = Y.loc[test]
    
    tree = Id5_Tree()
    tree.train(x_train, y_train)
    y_pred, acc = test_eval_tree(tree, x_test, y_test)
    fold_accs.append(acc)
    print(f'Acc#{fold} = {acc*100}%')
  
  print(f'5-Fold cross validation acc = {np.array(fold_accs).mean() * 100}%')


In [139]:
X, y = get_train_test_data(get_all=True)
# X.info()
# y.to_frame().info()
kFold_cross_validation(X, y)

Data_size = 374
Fold #1
Fold Size=299
Sample 1
Sample 2
Sample 3
Sample 4
Sample 8
Sample 9
Sample 10
Sample 11
Sample 13
Sample 15
Sample 16
Sample 17
Sample 20
Sample 21
Sample 22
Sample 23
Sample 24
Sample 25
Sample 26
Sample 27
Sample 28
Sample 29
Sample 30
Sample 31
Sample 32
Sample 33
Sample 34
Sample 35
Sample 36
Sample 37
Sample 38
Sample 39
Sample 40
Sample 41
Sample 43
Sample 44
Sample 45
Sample 46
Sample 47
Sample 48
Sample 49
Sample 50
Sample 51
Sample 52
Sample 53
Sample 54
Sample 55
Sample 56
Sample 57
Sample 58
Sample 59
Sample 60
Sample 61
Sample 62
Sample 64
Sample 65
Sample 67
Sample 69
Sample 70
Sample 71
Sample 72
Sample 73
Sample 74
Sample 75
Sample 76
Sample 77
Sample 78
Sample 80
Sample 81
Sample 84
Sample 85
Sample 86
Sample 87
Sample 88
Sample 89
Sample 92
Sample 93
Sample 94
Sample 95
Sample 97
Sample 98
Sample 100
Sample 101
Sample 102
Sample 104
Sample 105
Sample 106
Sample 109
Sample 110
Sample 111
Sample 112
Sample 113
Sample 114
Sample 115
Sample 116
Samp